In [ ]:
import pandas as pd
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_neo4j import Neo4jGraph, GraphCypherQAChain
from langchain_core.prompts import PromptTemplate

# Database connection
def connect():
    return Neo4jGraph(
        url="neo4j+s://02f54a39.databases.neo4j.io",
        username="neo4j",
        password="U9WSV67C8evx4nWCk48n3M0o7dX79T2XQ3cU1OJfP9c"
    )

# Initialize AI model
def init_model():
    return ChatGoogleGenerativeAI(
        model="gemini-2.5-flash",        
        temperature=0,
        convert_system_message_to_human=True
    )

# Format results if AI can't answer
def format_results(context):
    if not context:
        return "No results found."
    
    output = [f"\nFound {len(context)} results:\n"]
    
    for i, record in enumerate(context[:10], 1):
        if 'p' in record:
            place = record['p']
            output.append(f"{i}. {place.get('location', 'Unknown')}")
            output.append(f"   ID: {place.get('place_id')}, Coords: ({place.get('latitude')}, {place.get('longitude')})")
    
    if len(context) > 10:
        output.append(f"\n... and {len(context) - 10} more")
    
    return "\n".join(output)

# Main chat loop
def chat():
    print("🤖 Neo4j Chat Assistant")
    print("Ask questions about places in the database. Type 'quit' to exit.\n")
    
    graph = connect()
    llm = init_model()
    
    # Conversation history for context enrichment
    chat_history = []
    # Store the most recent full context returned by the chain
    last_context_records = []
    
    while True:
        question = input("You: ").strip()
        
        if question.lower() in ['quit', 'exit', 'q']:
            # Create DataFrame from the last available context for geo visualization
            geojson_df = pd.json_normalize(last_context_records) if last_context_records else pd.DataFrame()
            print("Goodbye! 👋")
            return geojson_df
        
        if not question:
            continue
        
        try:
            # Add conversation context to the question
            if chat_history:
                context_text = "\n".join([f"Previous Q: {q}\nPrevious A: {a}" for q, a in chat_history[-2:]])
                enhanced_question = f"{context_text}\n\nCurrent question: {question}"
            else:
                enhanced_question = question
            
            # Create chain with enhanced QA prompt
            qa_template = """You are a helpful assistant answering questions about places in a database.

Question: {question}

Database results:
{context}

Provide a clear, concise answer based on the results. Extract specific information requested.

Answer:"""
            
            qa_prompt = PromptTemplate(
                input_variables=["question", "context"],
                template=qa_template
            )
            
            chain = GraphCypherQAChain.from_llm(
                llm=llm,
                graph=graph,
                qa_prompt=qa_prompt,
                allow_dangerous_requests=True,
                verbose=True,
                return_intermediate_steps=True
            )
            
            result = chain.invoke({"query": enhanced_question})
            answer = result['result']
            
            # Capture the latest context output for later visualization
            intermediate_steps = result.get('intermediate_steps', [])
            for step in intermediate_steps:
                context_records = step.get('context')
                if context_records:
                    last_context_records = context_records
            
            # If AI doesn't know, format manually
            if "don't know" in answer.lower():
                if intermediate_steps:
                    answer = format_results(last_context_records)
            
            print(f"\nAssistant: {answer}\n")
            
            # Save to history (store original question, not enhanced)
            chat_history.append((question, answer))
            
        except Exception as e:
            print(f"\n❌ Error: {str(e)}\n")

if __name__ == "__main__":
    geojson_df = chat()

🤖 Neo4j Chat Assistant
Ask questions about places in the database. Type 'quit' to exit.



> Entering new GraphCypherQAChain chain...


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 250
Please retry in 20.925808285s. [links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 250
}
, retry_delay {
  seco


❌ Error: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 250
Please retry in 18.472470699s. [links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 250
}
, retry_delay {
  seconds: 18
}
]



In [4]:
geojson_df

,p.updated_at,p.latitude,p.created_at,p.location,p.place_id,p.longitude
0,2024-06-24,48.213716,2024-06-24,"Vienna, Austria",1797,16.382669
1,2024-06-24,48.213716,2024-06-24,"Vienna, Austria",1798,16.382669
2,2024-06-24,48.213716,2024-06-24,"Vienna, Austria",1799,16.382669
3,2024-06-24,48.213716,2024-06-24,"Vienna, Austria",1800,16.382669
4,2024-06-24,48.213716,2024-06-24,"Vienna, Austria",1802,16.382669
5,2024-07-08,48.202121,2024-07-08,"Vienna, Austria",2748,16.365529


In [5]:
import folium

# Make sure geojson_df has latitude and longitude columns
# Create a folium map centered on the mean latitude and longitude
center_lat = geojson_df['p.latitude'].mean()
center_lon = geojson_df['p.longitude'].mean()
m = folium.Map(location=[center_lat, center_lon], zoom_start=6)

# Add each point in geojson_df to the map
for idx, row in geojson_df.iterrows():
    folium.Marker(
        [row['p.latitude'], row['p.longitude']],
        popup=row['p.location']
    ).add_to(m)

# Display map in notebook
m

In [1]:
import mapbox
import pandas as pd
import json
from neo4j import GraphDatabase
import folium

def upload_and_visualize_neo4j_on_mapbox():
    # Connect to Neo4j
    driver = GraphDatabase.driver(
        "neo4j+s://02f54a39.databases.neo4j.io",
        auth=("neo4j", "U9WSV67C8evx4nWCk48n3M0o7dX79T2XQ3cU1OJfP9c")
    )
    # Query for spatial/location data
    with driver.session() as session:
        result = session.run("""
        MATCH (p:Place)
        RETURN p.latitude AS latitude, p.longitude AS longitude, p.location AS location, p.place_id AS place_id
        """)
        df = pd.DataFrame([dict(record) for record in result])

    # Prepare GeoJSON
    features = []
    for _, row in df.iterrows():
        feature = {
            "type": "Feature",
            "geometry": {
                "type": "Point",
                "coordinates": [row["longitude"], row["latitude"]],
            },
            "properties": {
                "place_id": row["place_id"],
                "location": row["location"]
            }
        }
        features.append(feature)
    geojson = {"type": "FeatureCollection", "features": features}

    # Upload to Mapbox as a dataset
    service = mapbox.Uploader(
        access_token="pk.eyJ1Ijoic2FoaWx5b3VzYWZwIiwiYSI6ImNtMzRkNTVoZDFkOWsycXI2c2EwZnlxdmQifQ.6DlxeBmkYtvH4drneaW4Cw"
    )
    resp = service.upload(
        bytes(json.dumps(geojson), encoding="utf-8"),
        "neo4j-places-dataset.geojson"
    )
    print(f"Upload to Mapbox status: {resp.status_code}, {resp.text}")

    # Visualize on a folium map (using Mapbox tiles)
    if not df.empty:
        center_lat = df['latitude'].mean()
        center_lon = df['longitude'].mean()
        m = folium.Map(
            location=[center_lat, center_lon],
            zoom_start=6,
            tiles='https://api.mapbox.com/styles/v1/mapbox/streets-v11/tiles/{z}/{x}/{y}?access_token=pk.eyJ1Ijoic2FoaWx5b3VzYWZwIiwiYSI6ImNtMzRkNTVoZDFkOWsycXI2c2EwZnlxdmQifQ.6DlxeBmkYtvH4drneaW4Cw',
            attr='Mapbox'
        )
        for _, row in df.iterrows():
            folium.Marker(
                [row["latitude"], row["longitude"]],
                popup=row["location"]
            ).add_to(m)
        display(m)
    else:
        print("No data to visualize.")



In [2]:
import json

def download_db_as_geojson():
    # Connect to Neo4j
    graph = Neo4jGraph(
        url="neo4j+s://02f54a39.databases.neo4j.io",
        username="neo4j",
        password="U9WSV67C8evx4nWCk48n3M0o7dX79T2XQ3cU1OJfP9c"
    )
    # Get all nodes with location and coordinates, adapt Cypher query as needed
    query = """
    MATCH (p)
    WHERE exists(p.latitude) AND exists(p.longitude)
    RETURN p.place_id AS place_id, p.location AS location, p.latitude AS latitude, p.longitude AS longitude
    """
    df = graph.query(query, data_type="pandas")
    features = []
    for _, row in df.iterrows():
        feature = {
            "type": "Feature",
            "geometry": {
                "type": "Point",
                "coordinates": [row["longitude"], row["latitude"]],
            },
            "properties": {
                "place_id": row["place_id"],
                "location": row["location"]
            }
        }
        features.append(feature)
    geojson = {"type": "FeatureCollection", "features": features}
    with open("neo4j-database-export.geojson", "w", encoding="utf-8") as f:
        json.dump(geojson, f, ensure_ascii=False, indent=2)
    print("Database exported as neo4j-database-export.geojson")
